TO DO
- Develop code to handle no game / double game gameweeks (see, e.g. funtion get_was_home)

In [1]:
# number of latest gameweek
gameweek = 20
# check number of total players from fpl's site
total_players = 11093653

In [2]:
import pandas as pd
import numpy as np

import json
import requests
import sys

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 500)

In [3]:
sys.path.append('..')

In [4]:
import py_scripts.download_data as download_data

# Fetch FPL data

In [5]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

In [6]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
#fpl_data.set_index('id',inplace=True)
fpl_data['season'] = '2022-23'
fpl_data['name'] = fpl_data['first_name'] + ' ' + fpl_data['second_name']
# normalize spelling of names
fpl_data['name'] = fpl_data['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
#fpl_data['name'] = fpl_data['name'].map(player_name_dict).fillna(fpl_data['name'])
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
fpl_data['position'] = fpl_data['element_type'].map(position_dict)
# change team number to team name
fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]
fpl_data['GW'] = gameweek
fpl_data['value'] = fpl_data['now_cost']/10.0
fpl_data['selected'] = (fpl_data['selected_by_percent'].astype(float) / 100) * total_players
fpl_data['transfers_balance'] = fpl_data['transfers_in_event'] - fpl_data['transfers_out_event']
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,0.0,0,Cédric,0.0,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,Arsenal,3,2,6674,63,26934,141,0.0,0.5,Cédric,28,0,0,0,0,0,0,0,0,0,0,0,5,2.4,1.6,0.0,0.4,0,0.00000,0.00979,0.00979,0.04350,453,166,415,149,508,205,458,166,NaN,,NaN,,NaN,,0.00000,0.0,0.03147,0.03147,0.13982,0.00,573,158,484,189,402,149,406,153,0.00000,0.00000,2022-23,Cedric Alves Soares,DEF,20,4.2,11093.653,-78
1,NaN,NaN,84450,0,0,0,0,1,3,3.5,3.0,3,Granit,3.0,3,False,,None,50,84450.jpg,4.1,Xhaka,3.0,False,None,a,Arsenal,3,74,797527,3604,648083,19206,0.6,14.8,Xhaka,1526,3,4,8,14,0,0,0,3,0,0,7,313,370.0,380.3,289.0,104.0,18,1.96170,1.96543,3.92713,16.04520,59,25,30,24,63,41,35,24,NaN,,3.0,,NaN,,0.11570,0.0,0.11592,0.23161,0.94631,0.83,191,109,108,49,54,22,99,34,1.06160,0.47182,2022-23,Granit Xhaka,MID,20,5.0,332809.590,-15602
2,100.0,100.0,153256,0,0,-3,3,1,3,1.0,0.5,0,Mohamed,0.5,4,False,,2022-08-30T11:30:06.278675Z,42,153256.jpg,1.2,Elneny,0.8,False,None,a,Arsenal,3,6,189544,665,245545,4164,0.1,1.4,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00000,0.04505,0.04505,1.29490,441,196,394,189,512,202,442,199,NaN,,NaN,,NaN,,0.00000,0.0,0.03653,0.03653,1.04992,1.62,581,302,315,146,372,161,202,60,0.81081,0.00000,2022-23,Mohamed Elneny,MID,20,4.2,88749.224,-3499
3,NaN,NaN,156074,0,0,-3,3,0,2,0.7,0.2,0,Rob,0.2,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,5,6717,65,17397,192,0.0,1.2,Holding,15,0,0,0,0,0,0,0,0,0,0,0,17,3.4,0.1,0.0,0.3,0,0.00000,0.00022,0.00022,0.27790,449,164,455,170,513,208,464,169,NaN,,NaN,,NaN,,0.00000,0.0,0.00132,0.00132,1.66740,0.00,583,165,352,120,406,153,427,160,0.00000,0.00000,2022-23,Rob Holding,DEF,20,4.2,11093.653,-127
4,100.0,100.0,167199,0,0,-2,2,0,3,2.7,2.2,3,Thomas,2.2,6,False,,2022-09-25T09:00:06.484502Z,48,167199.jpg,3.4,Partey,0.5,False,None,a,Arsenal,3,51,94308,1647,117236,2090,0.5,10.6,Partey,1302,2,0,9,8,0,0,0,2,0,0,4,236,281.0,202.4,154.0,63.8,15,1.44710,0.75764,2.20474,13.43710,104,46,91,67,138,90,105,73,NaN,,NaN,,NaN,,0.10003,0.0,0.05237,0.15240,0.92883,0.55,272,163,142,59,106,52,253,83,1.03687,0.62212,2022-23,Thomas Partey,MID,20,4.8,55468.265,-443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,NaN,NaN,449133,0,0,0,0,0,3,-1.0,0

In [7]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home_team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away_team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
fpl_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
0,2292871,NaN,False,False,61,None,0,False,None,8,NaN,1,NaN,[],2,4,74971,Arsenal,Everton
1,2292870,NaN,False,False,62,None,0,False,None,5,NaN,3,NaN,[],3,2,74972,Bournemouth,Brighton
2,2292882,NaN,False,False,73,None,0,False,None,7,NaN,5,NaN,[],2,3,74983,Brighton,Crystal Palace
3,2292883,NaN,False,False,74,None,0,False,None,12,NaN,6,NaN,[],4,4,74984,Chelsea,Liverpool
4,2292874,NaN,False,False,65,None,0,False,None,16,NaN,11,NaN,[],2,2,74975,Leeds United,Nottingham Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2293185,38.0,False,False,376,2023-05-28T15:00:00Z,0,False,False,3,NaN,8,NaN,[],2,2,75286,Everton,Bournemouth
376,2293186,38.0,False,False,377,2023-05-28T15:00:00Z,0,False,False,18,NaN,11,NaN,[],3,2,75287,Leeds United,Tottenham
377,2293187,38.0,False,False,378,2023-05-28T15:00:00Z,0,False,False,19,NaN,10,NaN,[],2,3,75288,Leicester City,West Ham
378,2293188,38.0,False,False,379,2023-05-28T15:00:00Z,0,False,False,9,NaN,14,NaN,[],2,4,75289,Manchester Utd,Fulham


In [8]:
def get_opponent(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    # my_team is the home team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team), \
                                         'away_team']
    # my_team is the away team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team), \
                                         'home_team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) )].shape[0]!=1:
        player = my_row['name']
        print(f'Number of games not equal to one for {player} ({my_team})')
        return np.nan

    return opponent_team.values[0]

In [9]:
def get_kickoff_time(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    kickoff_time = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) ), 'kickoff_time']
    if kickoff_time.shape[0]==1:
        return kickoff_time.values[0]
    else:
        player = my_row['name']
        print(f'Kickoff_time shape not one for {player} ({my_team})')
        return np.nan

In [10]:
def get_was_home(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        return 1
    elif fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        return 0
    else:
        player = my_row['name']
        print(f'Error in get_was_home for {player} ({my_team})')
        return np.nan

In [11]:
fpl_data['opp_team_name'] = fpl_data.apply(lambda x: get_opponent(x, fpl_fixtures, gameweek), axis=1)
fpl_data['kickoff_time'] = fpl_data.apply(lambda x: get_kickoff_time(x, fpl_fixtures, gameweek), axis=1)
fpl_data['was_home'] = fpl_data.apply(lambda x: get_was_home(x, fpl_fixtures, gameweek), axis=1)
fpl_data

Number of games not equal to one for Vicente Guaita (Crystal Palace)
Number of games not equal to one for James Tomkins (Crystal Palace)
Number of games not equal to one for James McArthur (Crystal Palace)
Number of games not equal to one for Christian Benteke (Crystal Palace)
Number of games not equal to one for Joel Ward (Crystal Palace)
Number of games not equal to one for Nathaniel Clyne (Crystal Palace)
Number of games not equal to one for Luka Milivojevic (Crystal Palace)
Number of games not equal to one for Jordan Ayew (Crystal Palace)
Number of games not equal to one for Wilfried Zaha (Crystal Palace)
Number of games not equal to one for Jeffrey Schlupp (Crystal Palace)
Number of games not equal to one for Will Hughes (Crystal Palace)
Number of games not equal to one for Jairo Riedewald (Crystal Palace)
Number of games not equal to one for Joachim Andersen (Crystal Palace)
Number of games not equal to one for Odsonne Edouard (Crystal Palace)
Number of games not equal to one for

Number of games not equal to one for Alfie Devine (Tottenham)
Number of games not equal to one for Troy Parrott (Tottenham)
Number of games not equal to one for Richarlison de Andrade (Tottenham)
Number of games not equal to one for Clement Lenglet (Tottenham)
Number of games not equal to one for Djed Spence (Tottenham)
Number of games not equal to one for Bryan Gil Salvatierra (Tottenham)
Kickoff_time shape not one for Vicente Guaita (Crystal Palace)
Kickoff_time shape not one for James Tomkins (Crystal Palace)
Kickoff_time shape not one for James McArthur (Crystal Palace)
Kickoff_time shape not one for Christian Benteke (Crystal Palace)
Kickoff_time shape not one for Joel Ward (Crystal Palace)
Kickoff_time shape not one for Nathaniel Clyne (Crystal Palace)
Kickoff_time shape not one for Luka Milivojevic (Crystal Palace)
Kickoff_time shape not one for Jordan Ayew (Crystal Palace)
Kickoff_time shape not one for Wilfried Zaha (Crystal Palace)
Kickoff_time shape not one for Jeffrey Schlu

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,0.0,0,Cédric,0.0,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,Arsenal,3,2,6674,63,26934,141,0.0,0.5,Cédric,28,0,0,0,0,0,0,0,0,0,0,0,5,2.4,1.6,0.0,0.4,0,0.00000,0.00979,0.00979,0.04350,453,166,415,149,508,205,458,166,NaN,,NaN,,NaN,,0.00000,0.0,0.03147,0.03147,0.13982,0.00,573,158,484,189,402,149,406,153,0.00000,0.00000,2022-23,Cedric Alves Soares,DEF,20,4.2,11093.653,-78,Tottenham,2023-01-15T16:30:00Z,0
1,NaN,NaN,84450,0,0,0,0,1,3,3.5,3.0,3,Granit,3.0,3,False,,None,50,84450.jpg,4.1,Xhaka,3.0,False,None,a,Arsenal,3,74,797527,3604,648083,19206,0.6,14.8,Xhaka,1526,3,4,8,14,0,0,0,3,0,0,7,313,370.0,380.3,289.0,104.0,18,1.96170,1.96543,3.92713,16.04520,59,25,30,24,63,41,35,24,NaN,,3.0,,NaN,,0.11570,0.0,0.11592,0.23161,0.94631,0.83,191,109,108,49,54,22,99,34,1.06160,0.47182,2022-23,Granit Xhaka,MID,20,5.0,332809.590,-15602,Tottenham,2023-01-15T16:30:00Z,0
2,100.0,100.0,153256,0,0,-3,3,1,3,1.0,0.5,0,Mohamed,0.5,4,False,,2022-08-30T11:30:06.278675Z,42,153256.jpg,1.2,Elneny,0.8,False,None,a,Arsenal,3,6,189544,665,245545,4164,0.1,1.4,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00000,0.04505,0.04505,1.29490,441,196,394,189,512,202,442,199,NaN,,NaN,,NaN,,0.00000,0.0,0.03653,0.03653,1.04992,1.62,581,302,315,146,372,161,202,60,0.81081,0.00000,2022-23,Mohamed Elneny,MID,20,4.2,88749.224,-3499,Tottenham,2023-01-15T16:30:00Z,0
3,NaN,NaN,156074,0,0,-3,3,0,2,0.7,0.2,0,Rob,0.2,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,5,6717,65,17397,192,0.0,1.2,Holding,15,0,0,0,0,0,0,0,0,0,0,0,17,3.4,0.1,0.0,0.3,0,0.00000,0.00022,0.00022,0.27790,449,164,455,170,513,208,464,169,NaN,,NaN,,NaN,,0.00000,0.0,0.00132,0.00132,1.66740,0.00,583,165,352,120,406,153,427,160,0.00000,0.00000,2022-23,Rob Holding,DEF,20,4.2,11093.653,-127,Tottenham,2023-01-15T16:30:00Z,0
4,100.0,100.0,167199,0,0,-2,2,0,3,2.7,2.2,3,Thomas,2.2,6,False,,2022-09-25T09:00:06.484502Z,48,167199.jpg,3.4,Partey,0.5,False,None,a,Arsenal,3,51,94308,1647,117236,2090,0.5,10.6,Partey,1302,2,0,9,8,0,0,0,2,0,0,4,236,281.0,202.4,154.0,63.8,15,1.44710,0.75764,2.20474,13.43710,104,46,91,67,138,90,105,73,NaN,,NaN,,NaN,,0.10003,0.0,0.05237,0.15240,0.92883,0.55,272,163,142,59,106,52,253,83,1.03687,0.62212,2022-23,Thomas Partey,MID,20,4.8,55468.265,-443,Tottenham,2023-01-15T16:30:00Z,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [12]:
fpl_data.to_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv')
fpl_fixtures.to_csv('../data/fpl_fixtures.csv')

In [13]:
difference_columns = [
    'total_points', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
    'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
    'penalties_saved', 'saves', 'threat', 'yellow_cards', 'red_cards'
]

In [14]:
def calculate_difference_to_previous_gameweek(my_row):
    my_id = my_row['id']
    if my_id in old_ids:
        for my_column in difference_columns:
            my_row[my_column] = (my_row[my_column] -
                                     fpl_previous.loc[fpl_previous['id']==my_id, my_column]).values[0]
        return my_row
    elif my_id in new_ids:
        # for new players, these should be the same already, but just in case
        my_row['total_points'] = my_row['event_points']
        return my_row
    else:
        print(f'Player id neither old or new: {my_id}')
        return np.nan

In [15]:
# create a csv that contains each week's fpl data in one table
if gameweek==1:
    fpl_data['total_points'] = fpl_data['event_points']
    fpl_data[fpl_data['minutes']>0].to_csv('../data/fpl_data.csv')
    print(f'Gameweek {gameweek} data saved.')
elif gameweek>1:
    fpl_previous = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek-1}.csv')
    new_ids = set(fpl_data['id']).difference(set(fpl_previous['id']))
    old_ids = set(fpl_data['id']).intersection(set(fpl_previous['id']))
    # convert all difference columns to floats
    fpl_data[difference_columns] = fpl_data[difference_columns].astype(float)
    fpl_gameweek = fpl_data.apply(lambda x: calculate_difference_to_previous_gameweek(x), axis=1)
    fpl_gameweek = fpl_gameweek[fpl_gameweek['minutes']>0]
    print(f'Number of players that played this week: {fpl_gameweek.shape[0]}')
    fpl_previous_weeks = pd.read_csv('../data/fpl_data.csv', index_col=0)
    print(f'Size of fpl_previous_weeks: {fpl_previous_weeks.shape[0]}')
    #fpl_previous_weeks = fpl_previous_weeks[fpl_previous_weeks['GW']<gameweek]
    fpl_all_weeks = pd.concat([fpl_previous_weeks, fpl_gameweek], axis=0)
    print(f'Size of fpl_all_weeks: {fpl_all_weeks.shape[0]}')
    display(fpl_all_weeks)
else:
    print('Check gameweek!')

Number of players that played this week: 309
Size of fpl_previous_weeks: 5237
Size of fpl_all_weeks: 5546


,id,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,starts_per_90,clean_sheets_per_90
0,3,NaN,NaN,84450,0,0,0,0,0,3,2.1,2.1,2,Granit,2.0,False,NaN,NaN,50,84450.jpg,2.0,Xhaka,0.7,False,NaN,a,Arsenal,3,2.0,8050,8050,8788,8788,0.4,0.4,Xhaka,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,16.6,15.0,2.0,3.4,98,32,81,52,167,90,123,65,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Granit Xhaka,MID,1,5.0,60582.963,-738,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,167199,0,0,0,0,0,3,2.1,2.1,3,Thomas,3.0,False,NaN,NaN,50,167199.jpg,3.0,Partey,0.7,False,NaN,a,Arsenal,3,3.0,11320,11320,7903,7903,0.6,0.6,Partey,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,22.6,12.9,10.0,4.6,61,23,95,58,96,57,85,48,NaN,NaN,4.0,NaN,NaN,NaN,2022-23,Thomas Partey,MID,1,5.0,60582.963,3417,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,184029,0,0,0,0,0,3,2.8,2.8,3,Martin,3.0,False,NaN,NaN,65,184029.jpg,3.0,Ødegaard,13.4,False,NaN,a,Arsenal,3,3.0,52695,52695,121593,121593,0.5,0.5,Ødegaard,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.4,8.3,23.0,3.5,220,96,116,68,52,29,119,64,3.0,NaN,1.0,NaN,NaN,NaN,2022-23,Martin degaard,MID,1,6.5,1159731.006,-68898,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,100.0,75.0,192895,0,0,0,0,0,2,2.8,2.1,1,Kieran,1.0,False,NaN,2022-08-04T15:00:05.276574Z,50,192895.jpg,1.0,Tierney,1.2,False,NaN,a,Arsenal,3,1.0,2682,2682,16585,16585,0.2,0.2,Tierney,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,0.2,0.0,0.6,195,85,249,88,491,201,245,88,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Kieran Tierney,DEF,1,5.0,103856.508,-13903,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,NaN,NaN,198869,0,0,0,0,0,2,2.4,2.4,5,Benjamin,5.0,False,NaN,NaN,45,198869.jpg,5.0,White,6.4,False,NaN,a,Arsenal,3,5.0,30058,30058,35607,35607,1.1,1.1,White,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25.0,31.4,12.0,0.0,4.3,36,10,99,32,349,150,90,24,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Ben White,DEF,1,4.5,553901.376,-5549,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [16]:
# save data
if gameweek>1:
    fpl_all_weeks.to_csv('../data/fpl_data.csv')

# Fetch fbref data

In [17]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
links = download_data.get_match_report_links(url)
links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League',
 'https:

In [18]:
# check which matches (links) are already collected
with open('../data/fbref_fixture_links.txt') as f:
    old_links = f.read().splitlines()
old_links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League',
 'https:

In [19]:
# get links to games for which data has not been collected yet
new_links = [link for link in links if link not in old_links]
new_links

['https://fbref.com/en/matches/ea58bf24/Aston-Villa-Leeds-United-January-13-2023-Premier-League',
 'https://fbref.com/en/matches/f0fd7541/Manchester-Derby-Manchester-United-Manchester-City-January-14-2023-Premier-League',
 'https://fbref.com/en/matches/11603f7f/Everton-Southampton-January-14-2023-Premier-League',
 'https://fbref.com/en/matches/20fbd1a1/Wolverhampton-Wanderers-West-Ham-United-January-14-2023-Premier-League',
 'https://fbref.com/en/matches/95d3b457/Nottingham-Forest-Leicester-City-January-14-2023-Premier-League',
 'https://fbref.com/en/matches/cfc2a193/Brighton-and-Hove-Albion-Liverpool-January-14-2023-Premier-League',
 'https://fbref.com/en/matches/3f992264/Brentford-Bournemouth-January-14-2023-Premier-League',
 'https://fbref.com/en/matches/564d2893/Chelsea-Crystal-Palace-January-15-2023-Premier-League',
 'https://fbref.com/en/matches/f93d11e7/Newcastle-United-Fulham-January-15-2023-Premier-League',
 'https://fbref.com/en/matches/4fc8b7b2/North-London-Derby-Tottenham-H

In [20]:
fbref_data = download_data.download_match_report_data(new_links)
# normalize spelling of player names
fbref_data[('General','Player')] = fbref_data[('General','Player')].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
# map certain player names such that they are the same for both fpl and fbref data
#fbref_data[('General','Player')] = fbref_data[('General','Player')].map(player_name_dict).fillna(fbref_data[('General','Player')])
fbref_data

100%|███████████████████████████████████████████| 12/12 [00:54<00:00,  4.55s/it]


General                                   Performance         \
                Player     #   Nation    Pos     Age Min         Gls Ast PK   
0        Ollie Watkins  11.0  eng ENG     FW  27-014  31           0   0  0   
1           Danny Ings   9.0  eng ENG     FW  30-174  59           0   0  0   
2          Emi Buendia  10.0   ar ARG  FW,AM  26-019  80           1   0  0   
3    Philippe Coutinho  23.0   br BRA     LM  30-215  10           0   0  0   
4         Jacob Ramsey  41.0  eng ENG  LM,LW  21-230  80           0   0  0   
..                 ...   ...      ...    ...     ...  ..         ...  .. ..   
352         Ben Davies  33.0  wls WAL     CB  29-270  77           0   0  0   
353    Clement Lenglet  34.0   fr FRA     CB  27-216  13           0   0  0   
354          Eric Dier  15.0  eng ENG     CB  29-004  90           0   0  0   
355    Cristian Romero  17.0   ar ARG     CB  24-267  90           0   0  0   
356        Hugo Lloris   1.0   fr FRA     GK  36-024  90           0   0  0   

                                                  Expected           SCA      \
    PKatt Sh SoT CrdY CrdR Touches Tkl Int Blocks       xG npxG  xAG SCA GCA   
0       0  0   0    0    0       6   0   0      0      0.0  0.0  0.0   0   0   
1       0  2   1    0    0      26   2   0      1      0.4  0.4  0.0   1   0   
2       0  1   1    0    0      31   2   1      0      0.3  0.3  0.0   0   0   
3       0  1   0    0    0      10   0   0      0      0.0  0.0  0.0   0   0   
4       0  0   0    0    0      32   1   1      0      0.0  0.0  0.1   4   0   
..    ... ..  ..  ...  ...     ...  ..  ..    ...      ...  ...  ...  ..  ..   
352     0  0   0    0    0      41   0   2      0      0.0  0.0  0.1   1   0   
353     0  0   0    0    0      20   0   0      1      0.0  0.0  0.0   0   0   
354     0  0   0    0    0      65   1   2      2      0.0  0.0  0.0   0   0   
355     0  1   0    1    0      62   4   0      1      0.0  0.0  0.0   0   0   
356     0  0   0    0    0      37   0   0      0      0.0  0.0  0.0   0   0   

    Passes                Dribbles          General                            \
       Cmp Att  Cmp% Prog     Succ Att        Squad HomeTeam         Opponent   
0        1   2  50.0    0        0   1  Aston Villa     True     Leeds United   
1        7  13  53.8    0        0   0  Aston Villa     True     Leeds United   
2       16  21  76.2    2        1   1  Aston Villa     True     Leeds United   
3        5   9  55.6    0        1   1  Aston Villa     True     Leeds United   
4       22  26  84.6    4        0   1  Aston Villa     True     Leeds United   
..     ...  ..   ...  ...      ...  ..          ...      ...              ...   
352     31  37  83.8    1        0   0    Tottenham    False  Manchester City   
353     15  18  83.3    3        0   0    Tottenham    False  Manchester City   
354     49  58  84.5    2        0   0    Tottenham    False  Manchester City   
355     44  50  88.0    0        0   0    Tottenham    False  Manchester City   
356     26  35  74.3    0        0   0    Tottenham    False  Manchester City   

    Total                           Short            Medium            Long  \
      Cmp Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   Cmp%  Cmp   
0       1   2  50.0       8       0     1   2   50.0      0   0    NaN    0   
1       7  13  53.8      55       0     4   6   66.7      1   1  100.0    0   
2      16  21  76.2     228      83     9  12   75.0      1   2   50.0    3   
3       5   9  55.6      72       7     4   7   57.1      0   0    NaN    1   
4      22  26  84.6     368      60    12  12  100.0      7   9   77.8    2   
..    ...  ..   ...     ...     ...   ...  ..    ...    ...  ..    ...  ...   
352    31  37  83.8     516     141    11  13   84.6     13  14   92.9    3   
353    15  18  83.3     268     178     9  10   90.0      3   4   75.0    3   
354    49  58  84.5     933     397    18  21   85.7     23  23  100.0    7   
355    44  50  88.0    

In [21]:
# change column name to align the new Opta data with the old Statsbomb data
fbref_data.columns = fbref_data.columns.values
fbref_data.columns = pd.MultiIndex.from_tuples(fbref_data.rename(columns={('Expected', 'xAG'): ('Expected', 'xA')}))
fbref_data

General                                   Performance         \
                Player     #   Nation    Pos     Age Min         Gls Ast PK   
0        Ollie Watkins  11.0  eng ENG     FW  27-014  31           0   0  0   
1           Danny Ings   9.0  eng ENG     FW  30-174  59           0   0  0   
2          Emi Buendia  10.0   ar ARG  FW,AM  26-019  80           1   0  0   
3    Philippe Coutinho  23.0   br BRA     LM  30-215  10           0   0  0   
4         Jacob Ramsey  41.0  eng ENG  LM,LW  21-230  80           0   0  0   
..                 ...   ...      ...    ...     ...  ..         ...  .. ..   
352         Ben Davies  33.0  wls WAL     CB  29-270  77           0   0  0   
353    Clement Lenglet  34.0   fr FRA     CB  27-216  13           0   0  0   
354          Eric Dier  15.0  eng ENG     CB  29-004  90           0   0  0   
355    Cristian Romero  17.0   ar ARG     CB  24-267  90           0   0  0   
356        Hugo Lloris   1.0   fr FRA     GK  36-024  90           0   0  0   

                                                  Expected           SCA      \
    PKatt Sh SoT CrdY CrdR Touches Tkl Int Blocks       xG npxG   xA SCA GCA   
0       0  0   0    0    0       6   0   0      0      0.0  0.0  0.0   0   0   
1       0  2   1    0    0      26   2   0      1      0.4  0.4  0.0   1   0   
2       0  1   1    0    0      31   2   1      0      0.3  0.3  0.0   0   0   
3       0  1   0    0    0      10   0   0      0      0.0  0.0  0.0   0   0   
4       0  0   0    0    0      32   1   1      0      0.0  0.0  0.1   4   0   
..    ... ..  ..  ...  ...     ...  ..  ..    ...      ...  ...  ...  ..  ..   
352     0  0   0    0    0      41   0   2      0      0.0  0.0  0.1   1   0   
353     0  0   0    0    0      20   0   0      1      0.0  0.0  0.0   0   0   
354     0  0   0    0    0      65   1   2      2      0.0  0.0  0.0   0   0   
355     0  1   0    1    0      62   4   0      1      0.0  0.0  0.0   0   0   
356     0  0   0    0    0      37   0   0      0      0.0  0.0  0.0   0   0   

    Passes                Dribbles          General                            \
       Cmp Att  Cmp% Prog     Succ Att        Squad HomeTeam         Opponent   
0        1   2  50.0    0        0   1  Aston Villa     True     Leeds United   
1        7  13  53.8    0        0   0  Aston Villa     True     Leeds United   
2       16  21  76.2    2        1   1  Aston Villa     True     Leeds United   
3        5   9  55.6    0        1   1  Aston Villa     True     Leeds United   
4       22  26  84.6    4        0   1  Aston Villa     True     Leeds United   
..     ...  ..   ...  ...      ...  ..          ...      ...              ...   
352     31  37  83.8    1        0   0    Tottenham    False  Manchester City   
353     15  18  83.3    3        0   0    Tottenham    False  Manchester City   
354     49  58  84.5    2        0   0    Tottenham    False  Manchester City   
355     44  50  88.0    0        0   0    Tottenham    False  Manchester City   
356     26  35  74.3    0        0   0    Tottenham    False  Manchester City   

    Total                           Short            Medium            Long  \
      Cmp Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   Cmp%  Cmp   
0       1   2  50.0       8       0     1   2   50.0      0   0    NaN    0   
1       7  13  53.8      55       0     4   6   66.7      1   1  100.0    0   
2      16  21  76.2     228      83     9  12   75.0      1   2   50.0    3   
3       5   9  55.6      72       7     4   7   57.1      0   0    NaN    1   
4      22  26  84.6     368      60    12  12  100.0      7   9   77.8    2   
..    ...  ..   ...     ...     ...   ...  ..    ...    ...  ..    ...  ...   
352    31  37  83.8     516     141    11  13   84.6     13  14   92.9    3   
353    15  18  83.3     268     178     9  10   90.0      3   4   75.0    3   
354    49  58  84.5     933     397    18  21   85.7     23  23  100.0    7   
355    44  50  88.0    

In [22]:
# save links that have been downloaded already so that in the future one can only fetch data from new games
with open('../data/fbref_fixture_links.txt', 'w') as f:
    f.write('\n'.join(links))

In [23]:
old_fbref_data = pd.read_csv('../data/fbref_data.csv', index_col=0, header=[0,1])
#old_fbref_data.to_csv(f'../data/fbref_data_backup_week{gameweek}.csv')

In [24]:
fbref_data = pd.concat([old_fbref_data, fbref_data], axis=0)
print(f'Number of rows in fbref_data: {fbref_data.shape[0]}.')

Number of rows in fbref_data: 5618.


In [25]:
fbref_data.to_csv(f'../data/fbref_data.csv')

## 22-23 Match results

In [26]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
data = pd.read_html(url)
results = data[0].dropna(axis=0, how='all').copy()
results

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,Match Report,NaN
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,38.0,Sun,2023-05-28,15:00,Everton,NaN,NaN,NaN,Bournemouth,NaN,Goodison Park,NaN,Head-to-Head,NaN
416,38.0,Sun,2023-05-28,15:00,Leicester City,NaN,NaN,NaN,West Ham,NaN,King Power Stadium,NaN,Head-to-Head,NaN
417,38.0,Sun,2023-05-28,15:00,Aston Villa,NaN,NaN,NaN,Brighton,NaN,Villa Park,NaN,Head-to-Head,NaN
418,38.0,Sun,2023-05-28,15:00,Leeds United,NaN,NaN,NaN,Tottenham,NaN,Elland Road,NaN,Head-to-Head,NaN


In [27]:
# fix Nott Forest team name
my_dict = {"Nott'ham Forest":'Nottingham Forest'}
results['Home'] = results['Home'].map(my_dict).fillna(results['Home'])
results['Away'] = results['Away'].map(my_dict).fillna(results['Away'])

In [28]:
results.to_csv('../data/PL_results_season22_23.csv')